In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
with open('faults.csv') as csvfile:
    temp_csvreader = csv.reader(csvfile)
    temp_data = pd.DataFrame(temp_csvreader)
temp_data.rename(columns=temp_data.iloc[0]).drop(temp_data.index[0])

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
1,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
2,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
3,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.215,1,0,0,0,0,0,0
4,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
5,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1938,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1939,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1940,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [3]:
# 분량 철판 데이터 불러오기
with open('faults.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None)
    rows = []
    for row in csvreader:
        rows.append(row)

input_cnt, output_cnt = 27,7
data = np.asarray(rows,dtype = 'float32')
data

array([[4.200000e+01, 5.000000e+01, 2.709000e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [6.450000e+02, 6.510000e+02, 2.538079e+06, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [8.290000e+02, 8.350000e+02, 1.553913e+06, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [1.450000e+02, 1.740000e+02, 3.867790e+05, ..., 0.000000e+00,
        0.000000e+00, 1.000000e+00],
       [1.370000e+02, 1.700000e+02, 4.224970e+05, ..., 0.000000e+00,
        0.000000e+00, 1.000000e+00],
       [1.261000e+03, 1.281000e+03, 8.795100e+04, ..., 0.000000e+00,
        0.000000e+00, 1.000000e+00]], dtype=float32)

In [65]:
# 초기 weight,bias 를 포함한 파라미터 및 하이퍼파라미터 설정
RND_MEAN = 0
RND_STD = 0.0030
LEARNING_RATE = 0.0001

weight = np.random.normal(RND_MEAN,RND_STD,[input_cnt,output_cnt])
bias = np.zeros([output_cnt])

epoch_count = 500
mb_size = 10
report = 1

In [66]:
# 학습데이터를 불러와 섞어서 출력해주고 테스트 데이터를 선택하면 테스트 데이터를 출력해주는 함수
def get_train_test_shuffle_data(data_type,mb_size,nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if data_type == 'train':
        if nth == 0:
            np.random.shuffle(shuffle_map[:test_begin_idx])
        result_x = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]][:,:-output_cnt]
        result_y = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]][:,-output_cnt:]
    elif data_type == 'test':
        result_x = data[shuffle_map[test_begin_idx:]][:,:-output_cnt]
        result_y = data[shuffle_map[test_begin_idx:]][:,-output_cnt:]
    else:
        print("data_type_error")
    
    return result_x,result_y

# 소프트맥스 함수와 크로스엔트로피 및 크로스엔트로피의 미분값을 산추해주는 함수
def softmax(x):
    max_elem = np.max(x,axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_cross_entropy_with_logits(x,y):
    probs = softmax(x)
    return -np.sum(y*np.log(probs + 1.0e-10),axis=1)

def softmax_cross_entropy_with_logits_derv(x,y):
    return softmax(x) - y

# 평가 함수
def eval_accuracy(output,y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)

    return np.mean(correct)

In [67]:
# 학습데이터를 분리하고 섞고 랜덤으로 추출하기 위한 인덱스와 섞음 인덱스 설정
shuffle_map = np.arange(data.shape[0])
np.random.shuffle(shuffle_map)
test_begin_idx = int(data.shape[0]*0.8)
step_count = int(data.shape[0]*0.8) // mb_size

# 데이터 학습 및 결과 도출
for epoch in range(epoch_count):
    losses, accs = [], []

    for n in range(step_count):
        train_x,train_y = get_train_test_shuffle_data('train',mb_size,n)

        output_train_X = np.matmul(train_x,weight) + bias
        entropy = softmax_cross_entropy_with_logits(output_train_X,train_y)
        loss = np.mean(entropy)
        accuracy = eval_accuracy(output_train_X,train_y)

        losses.append(loss)
        accs.append(accuracy)

        # backpropagation
        G_loss = 1.0

        g_loss_entropy = 1.0 / np.prod(entropy.shape)
        g_entorpy_output = softmax_cross_entropy_with_logits_derv(output_train_X,train_y)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entorpy_output * G_entropy

        g_outout_w = train_x.transpose()

        G_w = np.matmul(g_outout_w,G_output)
        G_b = np.sum(G_output, axis=0)

        weight -= LEARNING_RATE * G_w
        bias -= LEARNING_RATE * G_b
    
    if report > 0 and (epoch+1) % report == 0:
        test_x,test_y = get_train_test_shuffle_data('test',mb_size,n)
        output_test_x = np.matmul(test_x,weight) + bias
        acc = eval_accuracy(output_test_x,test_y)
        print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
              format(epoch+1,np.mean(losses),np.mean(accs),acc))

output_test_x = np.matmul(test_x,weight) + bias
final_acc = eval_accuracy(output_test_x,test_y)
print('\nFinal Test : final accuracy = {:5.3f}'.format(final_acc))      


Epoch 1: loss=16.460, accuracy=0.285/0.272
Epoch 2: loss=15.464, accuracy=0.328/0.337
Epoch 3: loss=15.925, accuracy=0.308/0.396
Epoch 4: loss=15.821, accuracy=0.313/0.329
Epoch 5: loss=15.375, accuracy=0.332/0.419
Epoch 6: loss=15.524, accuracy=0.326/0.247
Epoch 7: loss=15.286, accuracy=0.336/0.442
Epoch 8: loss=15.450, accuracy=0.329/0.391
Epoch 9: loss=15.643, accuracy=0.321/0.424
Epoch 10: loss=15.524, accuracy=0.326/0.401
Epoch 11: loss=15.851, accuracy=0.312/0.378
Epoch 12: loss=15.732, accuracy=0.317/0.427
Epoch 13: loss=16.148, accuracy=0.299/0.283
Epoch 14: loss=15.568, accuracy=0.324/0.283
Epoch 15: loss=15.494, accuracy=0.327/0.401
Epoch 16: loss=15.420, accuracy=0.330/0.260
Epoch 17: loss=15.791, accuracy=0.314/0.237
Epoch 18: loss=15.301, accuracy=0.335/0.393
Epoch 19: loss=15.598, accuracy=0.323/0.447
Epoch 20: loss=15.940, accuracy=0.308/0.398
Epoch 21: loss=15.598, accuracy=0.323/0.221
Epoch 22: loss=15.687, accuracy=0.319/0.422
Epoch 23: loss=15.494, accuracy=0.327/0.2